In [1]:
import sys
from tqdm import tqdm as tqdm
sys.path.insert(0,'/home/olgan96/.env/lib/python3.11/site-packages/')

In [2]:
import pandas as pd
import anndata

In [3]:
df_sig = pd.read_parquet('../../NO_BACKUP/tahoe/computed_signaturs_meta.parquet')

In [4]:
df_resp = pd.read_parquet('../../NO_BACKUP/pharmacodb/responses.parquet')

In [5]:
df_drugs = pd.read_parquet('../../NO_BACKUP/tahoe/drug_meta.parquet')

In [6]:
df_drugs = df_drugs[~df_drugs['pubchem_cid'].isna()].reset_index(drop=True)
df_drugs['pubchem_cid'] = df_drugs['pubchem_cid'].astype(int).astype(str)

In [7]:
pharm_cell_lines = pd.read_parquet('../../NO_BACKUP/pharmacodb/cell_lines.parquet')

In [8]:
pharm_comp = pd.read_parquet('../../NO_BACKUP/pharmacodb/compounds_pharmdb.parquet')

In [9]:
pharm_comp = pharm_comp.drop_duplicates(subset=['inchikey'])[['id', 'inchikey', 'pubchem']]

In [10]:
pharm_comp = pharm_comp.dropna()

In [11]:
pharm_comp

,id,inchikey,pubchem
1,2,JDYVLIMGUYDXMH-UHFFFAOYSA-N,375630
2,3,ZIBNPJVIVGAAMD-UHFFFAOYSA-N,378568
3,4,FRSPBOBJXMHBDQ-UHFFFAOYSA-N,378574
4,5,PQHMYYKJZLCSJD-UHFFFAOYSA-N,379229
5,6,WLHDJGPWRKGCGB-UHFFFAOYSA-N,370751
...,...,...,...
56144,56145,FBUCXDSCOIEFJY-UHFFFAOYSA-N,384578
56145,56146,BSQXMHXNJIAHEO-UHFFFAOYSA-N,378491
56146,56147,DIEODVBPTSOULE-UHFFFAOYSA-N,378488
56147,56148,UYBJQMWOCWMWQV-UHFFFAOYSA-N,378486


In [12]:
df_resp_ = df_resp.copy()

### overlapping

In [13]:
df_resp = df_resp_.merge(pharm_cell_lines[['id', 'accession_id']].dropna(), left_on='cell_line_id', right_on='id', how='left').rename(columns={'id_x': 'comp_id', 'idy':'cell_id'})

In [14]:
df_resp = df_resp.merge(pharm_comp, left_on='comp_id', right_on='id', how='left')

In [15]:
df_resp = df_resp.dropna(subset=['pubchem']).reset_index(drop=True)

In [16]:
df_sig['cell_line'] = df_sig['cl_sm_dose'].str.split('_').str[:2].str.join('_')

In [17]:
df_sig['drug'] = df_sig['cl_sm_dose'].str.split('_').str[2:].str[:-1].str.join('_')

In [18]:
df_sig['dose'] = df_sig['cl_sm_dose'].str.split('_').str[2:].str[-1]

In [19]:
df_sig = df_sig.merge(df_drugs[['drug', 'pubchem_cid']], on='drug', how='left').dropna()

In [20]:
df_sig['cell_line_drug'] = df_sig['cell_line'] + '_' + df_sig['pubchem_cid']

In [21]:
df_resp['cell_line_drug'] = df_resp['accession_id'] + '_' + df_resp['pubchem']

In [23]:
df_resp = df_resp.dropna(subset=['cell_line_drug']).reset_index(drop=True)

In [24]:
df_resp

,exp_id,comp_id,uid,compound,tissue,HS,Einf,EC50,AAC,IC50,...,DSS3,dose_response,dataset,cell_line_id,id_y,accession_id,id,inchikey,pubchem,cell_line_drug
0,2,4285,PDBC00224,"1,3-Bis(2-chloroethyl)-1-nitrosourea",Adrenal Gland,4.000000,0.000000,39.184394,0.010690,39.184394,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0300...",GDSC2,3,3.0,CVCL_0542,4285.0,DLGOEMSEDOSKAD-UHFFFAOYSA-N,2578,CVCL_0542_2578
1,3,13662,PDBC00161,2-Pyridinecarboxylic acid,Adrenal Gland,4.000000,0.000000,19.277201,0.002530,19.277201,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0100...",GDSC2,3,3.0,CVCL_0542,13662.0,SIOXPEMLGUPBBT-UHFFFAOYSA-N,1018,CVCL_0542_1018
2,4,15011,PDBC02174,2-fluoroAraA (fludarabine),Adrenal Gland,0.435226,100.000000,9.273881,0.000000,NaN,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0200...",GDSC2,3,3.0,CVCL_0542,15011.0,HBUBKKRHXORPQB-FJFJXFQQSA-N,657237,CVCL_0542_657237
3,5,23375,PDBC00173,5-Fluorouracil,Adrenal Gland,0.672320,100.000000,0.589045,0.000000,NaN,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.125,...",GDSC1,3,3.0,CVCL_0542,23375.0,GHASVSINZRGABV-UHFFFAOYSA-N,3385,CVCL_0542_3385
4,6,23375,PDBC00173,5-Fluorouracil,Adrenal Gland,0.286536,90.411958,0.009333,0.072408,NaN,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0632...",GDSC2,3,3.0,CVCL_0542,23375.0,GHASVSINZRGABV-UHFFFAOYSA-N,3385,CVCL_0542_3385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590208,6307645,53781,PDBC00117,Z-LLNle-CHO,Vulva/Vagina,4.000000,0.664130,1.020347,0.165900,1.023764,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.01, ...",GDSC1,1706,1706.0,CVCL_1733,53781.0,RNPDUXVFGTULLP-ATTQYFOMSA-N,16760646,CVCL_1733_16760646
5590209,6307646,53782,PDBC00137,Z-Leu-leu-leu-al,Vulva/Vagina,4.000000,0.000000,0.848934,0.048394,0.848934,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0039...",GDSC1,1706,1706.0,CVCL_1733,53782.0,TZYWCYJVHRLUCT-VABKMULXSA-N,462382,CVCL_1733_462382
5590210,6307647,53843,PDBC00043,ZM-447439,Vulva/Vagina,1.722408,0.000000,5.507571,0.047652,5.507571,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0156...",GDSC1,1706,1706.0,CVCL_1733,53843.0,OGNYUTNQZVRGMN-UHFFFAOYSA-N,9914412,CVCL_1733_9914412
5590211,6307650,55749,PDBC02281,kb NB 142-70,Vulva/Vagina,4.000000,58.528669,8.152642,0.067623,NaN,...,None,"[{'__typename': 'DoseResponse', 'dose': 0.0781...",GDSC1,1706,1706.0,CVCL_1733,55749.0,DHUAGGSHTKPOHU-UHFFFAOYSA-N,45258277,CVCL_1733_45258277


In [26]:
df_resp_overlap = df_resp[df_resp['cell_line_drug'].isin(df_sig['cell_line_drug'].unique())]

In [29]:
df_resp_overlap.to_parquet('../../NO_BACKUP/tahoe/responses_overlap.parquet', index=False)

In [28]:
df_sig_overlap = df_sig[df_sig['cell_line_drug'].isin(df_resp['cell_line_drug'].unique())]

In [30]:
df_sig_overlap.to_parquet('../../NO_BACKUP/tahoe/sig_overlap.parquet', index=False)